In [2]:
import gdown
import pandas as pd
import os
import csv
import re

In [3]:
def download_csv():
    sheet_url = "https://docs.google.com/spreadsheets/d/1vYD9nyEoI2CEWu2UY2xpedC_4mX3TUVD-XLCjOLphiA/export?format=csv"

    # Load the CSV into a DataFrame
    df = pd.read_csv(sheet_url)
    df.to_csv("form.csv")
    # print(df)

In [20]:
download_csv()

In [21]:
original_csv = "form.csv"
processed_csv = "processed.csv"
clothes_folder = "clothes_data"
possible_columns = [
    "服裝部位 Clothing Part",
    "上半身 衣服種類 Top Clothing Type",
    "下半身衣服種類 Bottom Clothing Type",
]

In [22]:
df = pd.read_csv(original_csv)
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df2 = df.drop("時間戳記", axis=1)
for col in possible_columns:
    df2 = df2.drop(col, axis=1)

In [24]:
# get all feature
all_features = []
for index, row in df.iterrows():
    for col in possible_columns:
        str = df.loc[index, col]
        try:
            str = list(str.split(","))
            for s in str:
                s = s.rstrip()
                s = s.lstrip()
                if (
                    s == "針織衣 Sweater"
                    or s == "毛衣 Sweater"
                    or s == "針/織衣 Sweater"
                ):
                    s = "毛衣、針織衣 Sweater"
                if s == "西裝褲 dress pants":
                    str.append("褲子 pants")
                    s = "西裝褲、裙 dress pants、skirt"
                all_features.append(s)
        except:
            pass
all_features = set(all_features)
for f in all_features:
    print(f)
print(len(all_features))

洋裝 Dress
網紗 Mesh
休閒褲/裙 casual pants/skirt
牛仔褲/裙 jeans / demi skirt
工裝褲/裙 cargo pants/skirt
毛衣、針/織衣 Sweater
連帽衣 Hoodie
羽絨衣 Down coat / jacket
裙子 skirt
T-Shirt
百褶裙 pleated skirt
運動褲/裙 sweat pants / sports skirt
西裝 Suit
下半身 Bottom
背心 Vest
西裝褲、裙 dress pants、skirt
外套 Jacket
吊帶褲/裙 overalls/jumper skirt
運動材質 Sport Tops
Shirt
毛衣、針織衣 Sweater
上半身 Top
緊身褲 leggings
褲子 pants
24


In [25]:
for index, row in df.iterrows():
    for fea in all_features:
        has = 0
        for col in possible_columns:
            str = df.loc[index, col]
            try:
                str = list(str.split(","))
                for s in str:
                    s = s.rstrip()
                    s = s.lstrip()
                    if (
                        s == "針織衣 Sweater"
                        or s == "毛衣 Sweater"
                        or s == "針/織衣 Sweater"
                    ):
                        s = "毛衣、針織衣 Sweater"
                    if s == "西裝褲 dress pants":
                        str.append("褲子 pants")
                        s = "西裝褲、裙 dress pants、skirt"
                    if s == fea:
                        has = 1
            except:
                pass
        df2.loc[index, fea] = has


print(df2)

                                             照片 Picture     品牌 Brand  \
0     https://drive.google.com/open?id=1Ki8gALrL6WEy...          H&M   
1     https://drive.google.com/open?id=1rsB7XhHjAww6...          H&M   
2     https://drive.google.com/open?id=1BEUPTgjmZ0uZ...          H&M   
3     https://drive.google.com/open?id=12MW0xvZVVNVo...          H&M   
4     https://drive.google.com/open?id=1rj0PNtlTPp5O...          H&M   
...                                                 ...          ...   
1408  https://drive.google.com/open?id=1J4cTA_SGx-dl...  New Balance   
1409  https://drive.google.com/open?id=1720Blp7ssTFe...  New Balance   
1410  https://drive.google.com/open?id=1xsdvpRxTNXOn...  New Balance   
1411  https://drive.google.com/open?id=13pqfc9SdibC_...  New Balance   
1412  https://drive.google.com/open?id=1hC-fezhwHUuS...  New Balance   

     衣服長度 Clothes Length  價格 Price  洋裝 Dress  網紗 Mesh  \
0                短 short       499       0.0      0.0   
1                 長 l

In [26]:
for index, row in df2.iterrows():
    image_url = row["照片 Picture"]  # 從表單中的照片 URL 讀取

    if re.search(r"open\?id=", image_url):
        output = f"image_{index:05}.jpg"
        df2.loc[index, "照片 Picture"] = output
df2.to_csv(processed_csv)

# check image downloaded or not

In [ ]:
def download_image(image_url, path):
    file_id = image_url.split("open?id=")[-1]
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    gdown.download(url, path, quiet=False)

In [ ]:
for index, row in df2.iterrows():
    output = f"image_{index:05}.jpg"
    path = os.path.join(clothes_folder, output)
    if not os.path.exists(path):
        image_url = df.loc[index, "照片 Picture"]
        print("download ", index, " from ", image_url)
        download_image(image_url, path)

download  0  from  https://drive.google.com/open?id=1Ki8gALrL6WEyGeBkwZQdO5J5pfnufz37


Downloading...
From: https://drive.google.com/uc?export=download&id=1Ki8gALrL6WEyGeBkwZQdO5J5pfnufz37
To: c:\Users\doo12\OneDrive\桌面\ML\finalProject\ClothesPredicting\clothes_data\image_00000.jpg
100%|██████████| 19.3k/19.3k [00:00<00:00, 472kB/s]


download  1  from  https://drive.google.com/open?id=1rsB7XhHjAww6w9RjucSgiJSnkEO5r_PL


Downloading...
From: https://drive.google.com/uc?export=download&id=1rsB7XhHjAww6w9RjucSgiJSnkEO5r_PL
To: c:\Users\doo12\OneDrive\桌面\ML\finalProject\ClothesPredicting\clothes_data\image_00001.jpg
100%|██████████| 29.4k/29.4k [00:00<00:00, 865kB/s]


download  2  from  https://drive.google.com/open?id=1BEUPTgjmZ0uZS5Llj5Btd8xOw-oxyeuW


...

download  3  from  https://drive.google.com/open?id=12MW0xvZVVNVoMt-YUGUnPVzg_v_Z5-F-


Downloading...
From: https://drive.google.com/uc?export=download&id=12MW0xvZVVNVoMt-YUGUnPVzg_v_Z5-F-
To: c:\Users\doo12\OneDrive\桌面\ML\finalProject\ClothesPredicting\clothes_data\image_00003.jpg
100%|██████████| 27.5k/27.5k [00:00<00:00, 399kB/s]


download  4  from  https://drive.google.com/open?id=1rj0PNtlTPp5O0Vur_aymMQkDBP4y7Iv6


Downloading...
From: https://drive.google.com/uc?export=download&id=1rj0PNtlTPp5O0Vur_aymMQkDBP4y7Iv6
To: c:\Users\doo12\OneDrive\桌面\ML\finalProject\ClothesPredicting\clothes_data\image_00004.jpg
100%|██████████| 18.7k/18.7k [00:00<00:00, 701kB/s]


download  5  from  https://drive.google.com/open?id=1_yRapskmxZj4dERJUO5pND4RjambTjBh
